In [6]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_excel('../dashboard/data/notaclimat.xlsx')#, dtype={'Groupe': str})


In [13]:
import pandas as pd
df = pd.read_excel('../dashboard/data/notaclimat.xlsx')#, dtype={'Groupe': str})
df = df.drop([0])
companies = df.iloc[:, 0:1].rename(columns={'Groupe':'company_name'})#.astype(str)
companies

# create IDs

import base64

def encode(text):
  btext = text.encode('utf-8')[:6]
  return base64.b64encode(btext).decode("utf-8") 


companies['company_id'] = [encode(x) for x in companies['company_name']]
companies = companies[['company_id', 'company_name']]

In [14]:
companies

,company_id,company_name
1,QW5kcm9z,Andros
2,QXNzb2Np,Associated British Foods (alim.)
3,QmVs,Bel
4,RGFub25l,Danone
5,RWNvdG9u,Ecotone
6,RmVycmVy,Ferrero
7,R2VuZXJh,General Mills
8,SGVybw==,Hero
9,S2VsbG9n,Kellogg
10,TWFycw==,Mars


In [15]:
df

,Groupe,Logo groupe,Drapeau pays,CA,Top marque 1,Top marque 2,Top marque 3,Top marque 4,Top marque 5,Logo marque 1,...,Scope 3.6,Scope 3.7,Scope 3.8,Scope 3.9,Scope 3.10,Scope 3.11,Scope 3.12,Scope 3.13,Scope 3.14,Scope 3.15
1,Andros,NaN,NaN,"2,5 Mrd € (est. 2020)",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Associated British Foods (alim.),NaN,NaN,4 Mrd € (2020) (groupe complet 16 Mrd €),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bel,NaN,NaN,"3,5 Mrd € (2020)",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Danone,NaN,NaN,25 Mrd € (2019),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Ecotone,NaN,NaN,650 Mio € (2019),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Ferrero,NaN,NaN,11 Mrd € (2019),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,General Mills,NaN,NaN,15 Mrd € (2020),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Hero,NaN,NaN,1 Mrd € (2020),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Kellogg,NaN,NaN,11 Mrd € (2019),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Mars,NaN,NaN,32 Mrd € (2020),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
